# Who Do You Sound Like?

---
## Section 1
### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import seaborn as sns

import os
import IPython.display as ipd

import spotipy as sp
import tekore as tk
import librosa as lib
import librosa.display as libd

---

**BELOW:** Importing master Spotify song dataset, sourced from [Kaggle](https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks?select=data.csv). Some brief descriptions of less tangible features, as defined by [Spotify](https://developer.spotify.com/documentation/web-api/reference/):
- **Acousticness:** A confidence measure from 0.0 to 1.0 of whether the track is acoustic.
- **Danceability:** How suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. 
- **Energy:** A measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
- **Instrumentalness:** Predicts whether a track contains vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.
- **Liveness:** Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.
- **Loudness:** The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typically range between -60 and 0 dB.
- **Popularity:** The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g., the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity.
- **Speechiness:** Detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g., talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.
- **Valence:** A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g., happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g., sad, depressed, angry).

In [2]:
df = pd.read_csv('../data_raw/kg_spotify_master_data.csv')
df.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [3]:
df.tail()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
170648,0.608,2020,0.08460,"['Anuel AA', 'Daddy Yankee', 'KAROL G', 'Ozuna...",0.786,301714,0.808,0,0KkIkfsLEJbrcIhYsCL7L5,0.000289,7,0.0822,-3.702,1,China,72,2020-05-29,0.0881,105.029
170649,0.734,2020,0.20600,['Ashnikko'],0.717,150654,0.753,0,0OStKKAuXlxA0fMH54Qs6E,0.000000,7,0.1010,-6.020,1,Halloweenie III: Seven Days,68,2020-10-23,0.0605,137.936
170650,0.637,2020,0.10100,['MAMAMOO'],0.634,211280,0.858,0,4BZXVFYCb76Q0Klojq4piV,0.000009,4,0.2580,-2.226,0,AYA,76,2020-11-03,0.0809,91.688
170651,0.195,2020,0.00998,['Eminem'],0.671,337147,0.623,1,5SiZJoLXp3WOl3J4C8IK0d,0.000008,2,0.6430,-7.161,1,Darkness,70,2020-01-17,0.3080,75.055
170652,0.642,2020,0.13200,"['KEVVO', 'J Balvin']",0.856,189507,0.721,1,7HmnJHfs0BkFzX4x8j0hkl,0.004710,7,0.1820,-4.928,1,Billetes Azules (with J Balvin),74,2020-10-16,0.1080,94.991


In [4]:
df.shape

(170653, 19)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170653 entries, 0 to 170652
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   valence           170653 non-null  float64
 1   year              170653 non-null  int64  
 2   acousticness      170653 non-null  float64
 3   artists           170653 non-null  object 
 4   danceability      170653 non-null  float64
 5   duration_ms       170653 non-null  int64  
 6   energy            170653 non-null  float64
 7   explicit          170653 non-null  int64  
 8   id                170653 non-null  object 
 9   instrumentalness  170653 non-null  float64
 10  key               170653 non-null  int64  
 11  liveness          170653 non-null  float64
 12  loudness          170653 non-null  float64
 13  mode              170653 non-null  int64  
 14  name              170653 non-null  object 
 15  popularity        170653 non-null  int64  
 16  release_date      17

In [6]:
# Dropping the "explicit" column, as this won't be relevant for audio analysis
df.drop(columns='explicit', inplace=True)

In [7]:
# According to Spotify, "liveness" predicts whether or not the track was recorded as part of a live show.
# Spotify's algorithm looks for audience noise in the track to make these predictions. A value > 0.8 suggests the track is a live recording.
# Here, isolating and dropping assumed "live" recordings, as their sonic qualities usually differ quite a bit from studio-recorded music.
df = df[(df['liveness'] <= 0.8)]

In [8]:
df.shape

(167244, 18)

In [9]:
df.columns

Index(['valence', 'year', 'acousticness', 'artists', 'danceability',
       'duration_ms', 'energy', 'id', 'instrumentalness', 'key', 'liveness',
       'loudness', 'mode', 'name', 'popularity', 'release_date', 'speechiness',
       'tempo'],
      dtype='object')

**ABOVE:** We can see that some songs have duplicate entries in the dataframe. Some reasons duplicates can legitimately occur include cover versions by other artists, single and album versions, different songs with the same name, and often with orchestral music, different recordings of the same piece by different ensembles.

However, I want to be sure not to include any duplicate songs that actually have identical data. **Below**, I'm using a simple `drop_duplicates()` method, factoring out the song's `id` (this is always unique on Spotify). I'll also investigate the other potentially problematic duplicate case, in which there are album and single versions of a song that are very similar, but don't have identical metrics. This can happen because of the way albums are formatted versus the way singles are formatted.

In [10]:
# Defining the subset of columns I'll pass into drop_duplicates()
no_id = ['name', 'artists', 'tempo', 'key', 'loudness', 'mode', 'duration_ms', 'energy', 'instrumentalness', 'speechiness', 
         'acousticness', 'danceability', 'valence', 'popularity', 'liveness', 'year', 'release_date']

df.drop_duplicates(subset=no_id, inplace=True)

In [11]:
df.shape

(166706, 18)

In [12]:
# Viewing duplicate song titles
#pd.concat(n for _, n in df.groupby('name') if len(n) > 1)

---

**BELOW**: Rounding `tempo` values to the nearest single decimal place. A few notes on this choice, and on tempo in general:
1. `tempo` is measured in beats per minute, and is generally expressed in integer values in the music world.
2. The decimal places exist in the original dataframe because the `tempo` metric is averaged for the entire song. Many songs recorded before the advent of digital recording and *click tracks* (a regular, metronomic pulse musicians hear in their headphones when recording to keep a steady pace) do not have a consistent tempo, due to natural human error in timing. 
3. Some old songs have also been altered by the idiosyncrasies of analog recording - an improperly aligned tape machine can cause small tempo fluctuations on playback, or when transferred to a new tape reel during the mastering process.
4. Ambient, orchestral, purely acoustic, purely vocal, and other such pieces without a strong pulse can also "trick" Spotify's tempo algorithm. Computers are largely able to infer tempo based on regular pulses (often in the attack sounds of drums and other rhythm instruments, known as *transients*) that machine learning algorithms can use as something like anchor points. Measuring the temporal distance between these "anchors" ideally yields a tempo.

In [13]:
# Rounding "tempo" to a single decimal place
df['tempo'] = round(df['tempo'], 1)

In [14]:
df['tempo'].value_counts()

120.0    916
130.0    702
128.0    699
100.0    685
140.0    651
        ... 
38.3       1
214.1      1
44.8       1
44.7       1
38.0       1
Name: tempo, Length: 1776, dtype: int64

As mentioned **above**, age can drastically affect an audio recording. In the early half of the 20th century, and especially in the 1920s and 1930s, recording technology was still very much in the process of maturing. The full frequency range of human hearing (20Hz to 20kHz) wasn't reproduceable with the microphones and media of the time, and additional noise, often in the form of white noise or electrical hum, was clearly audible. With this in mind, I'm going to drop entries for songs recorded prior to 1940, as they tend to suffer the most prominently from these issues. The excessive noise and poor sound reproduction of many of these recordings are likely to skew their metrics.

In [15]:
df['year'].value_counts()

2018    2090
2010    1989
2020    1988
1967    1980
2014    1980
        ... 
1925     268
1924     233
1923     182
1921     150
1922      68
Name: year, Length: 100, dtype: int64

In [16]:
df = df[(df['year'] >= 1940)]

---

In [17]:
# Modality (here defined as 0 for a minor key, 1 for a major key)
# Checking for entries where "mode" is equal to -1. This is the equivalent of a null or non-conclusive analysis.
# None found
df[df['mode'] == -1]

,valence,year,acousticness,artists,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo


In [18]:
# Key (here defined as 0-11, where 0 represents C, and keys increase chromatically with integer)
# Checking for entries where "key" is equal to -1. This is the equivalent of a null or non-conclusive analysis.
# None found
df[df['key'] == -1]

,valence,year,acousticness,artists,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo


In [19]:
df['year'].value_counts()

2018    2090
2010    1989
2020    1988
2014    1980
1967    1980
        ... 
1942    1631
1947    1618
1941     922
1944     737
1943     687
Name: year, Length: 81, dtype: int64

In [20]:
# Engineering a feature for "full_key", which contains the letter representing the key's fundamental pitch, and the modality,
# representing whether the key is major or minor.
df['key_letter'] = df['key'].map({0: 'C', 1: 'C#', 2: 'D', 3: 'D#', 4: 'E', 5: 'F', 6: 'F#', 7: 'G', 8: 'G#', 9: 'A', 10: 'A#', 11: 'B'})
df['maj_min'] = df['mode'].map({0: 'minor', 1: 'major'})

df['full_key'] = df['key_letter'] + ' ' + df['maj_min']

In [21]:
# Dropping columns engineered to make "full_key", as they're no longer necessary.
df.drop(columns=['key_letter', 'maj_min'], inplace=True)

---

In [22]:
# Reordering columns
df = df[['name', 'artists', 'tempo', 'key', 'mode', 'full_key', 'loudness', 'duration_ms', 'energy', 'instrumentalness', 'speechiness', 
         'acousticness', 'danceability', 'valence', 'popularity', 'liveness', 'year', 'release_date', 'id']]

In [23]:
df.head()

,name,artists,tempo,key,mode,full_key,loudness,duration_ms,energy,instrumentalness,speechiness,acousticness,danceability,valence,popularity,liveness,year,release_date,id
3606,Main Title - Gone With the Wind,['Max Steiner'],121.8,5,1,F major,-8.326,246773,0.430,0.854000,0.0357,0.680,0.170,0.171,33,0.459,1940,1940-01-17,6kucJsz2GR6oN8glzqGdTh
3607,Let's Do It (with Eddie Heywood & His Orchestr...,"['Billie Holiday', 'Eddie Heywood']",142.3,5,0,F minor,-14.450,175893,0.182,0.000171,0.0537,0.983,0.543,0.597,23,0.258,1940,1940,3LKwoKXJBpWBSKID58IgYw
3608,9:20 Special,['Count Basie'],198.7,2,0,D minor,-14.557,187400,0.333,0.089200,0.1010,0.871,0.513,0.876,27,0.297,1940,1940,1GoA6bkY0rZsc7aAHGlfl5
3609,Seven Come Eleven (feat. Benny Goodman & Charl...,"['Benny Goodman Sextet', 'Benny Goodman', 'Cha...",115.7,1,1,C# major,-6.975,165800,0.720,0.016000,0.0292,0.727,0.703,0.823,26,0.185,1940,1940,5SikmTT1Bf2M1NetWLeDyi
3610,"Final Speech (From ""The Great Dictator"")",['Charlie Chaplin'],124.4,2,1,D major,-8.429,222867,0.383,0.000000,0.6590,0.885,0.721,0.629,25,0.503,1940,1940,5HjcDXSfMbBmG4x04wELjK


---

In [24]:
df['popularity'].describe()

count    152342.000000
mean         34.349654
std          20.757630
min           0.000000
25%          20.000000
50%          36.000000
75%          50.000000
max         100.000000
Name: popularity, dtype: float64

In [25]:
# Based on the metrics immediately above, I'm going to drop all songs with a popularity rating of less than 10 out of 100.
# The mean popularity skews low, which makes sense based on how Spotify measures it (definition above). Even still, I'd like to filter
# out the least popular songs, given that the point of the project is to help new artists market themselves.
df = df[(df['popularity'] >= 10)]

In [28]:
df.shape

(126086, 19)

In [ ]:
# DROP CAST RECORDINGS, RE-INDEX DATAFRAME

In [45]:
df[df['artists'].str.contains('Cast', case=False)]

,name,artists,tempo,key,mode,full_key,loudness,duration_ms,energy,instrumentalness,speechiness,acousticness,danceability,valence,popularity,liveness,year,release_date,id
4773,Sea Murmurs (After. M. Castelnuovo-Tedesco) [T...,"['Jascha Heifetz', 'Mario Castelnuovo-Tedesco'...",131.4,1,1,C# major,-28.684,91733,0.084,0.064300,0.0506,0.9840,0.232,0.0384,11,0.1060,1945,1945-01-01,6NwVQYVrY73tRRLPr2V5FP
6545,The Pajama Game: Hey There,"['Richard Adler', 'Jerry Ross', 'John Raitt', ...",84.9,6,1,F# major,-12.420,212133,0.193,0.000000,0.0329,0.9670,0.267,0.1710,20,0.3180,1954,1954,1Fowkn0kUthBmHlq3dmJ2y
6562,The Pajama Game: I'm Not at All in Love,"['Richard Adler', 'Jerry Ross', 'Janis Paige',...",105.7,9,1,A major,-10.802,232067,0.347,0.000000,0.0434,0.7960,0.633,0.5330,19,0.5820,1954,1954,1S3aNCXOAVej9bAYhtIlAB
6585,The Pajama Game: Steam Heat,"['Richard Adler', 'Jerry Ross', 'Carol Haney',...",138.1,5,1,F major,-9.216,267507,0.565,0.000047,0.1790,0.6560,0.505,0.4930,18,0.4090,1954,1954,0SJ0T1EniphXlLAhwIUwgr
6592,The Pajama Game: Hernando's Hideaway,"['Richard Adler', 'Jerry Ross', 'Carol Haney',...",114.1,0,0,C minor,-12.370,228893,0.280,0.000001,0.0824,0.8510,0.688,0.4180,18,0.1170,1954,1954,2aPjqjiRQukYdSJLpsFiKF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169036,We Are Young (Glee Cast Version),['Glee Cast'],92.0,7,1,G major,-6.086,249773,0.555,0.000000,0.0498,0.0504,0.378,0.5240,55,0.1990,2012,2012-05-15,1zugfOhn6BebboqGcpUlba
169146,Surf Crazy,"['Spencer Lee', 'Keely Hawkes', 'Teen Beach Mo...",169.9,9,1,A major,-3.282,182707,0.953,0.000000,0.1060,0.1550,0.426,0.6780,51,0.1320,2013,2013-01-01,4Xt0OKNJtbyqx5GEX2XpVF
169189,Seasons Of Love (Glee Cast Version),['Glee Cast'],89.0,5,1,F major,-6.354,184480,0.507,0.000002,0.0259,0.6180,0.605,0.2530,55,0.1220,2013,2013-10-15,5gKfjl3XR7Oqf07gkjwLQk
169541,Good Good Father - Live,['Casting Crowns'],147.6,8,1,G# major,-8.402,379213,0.420,0.000000,0.0361,0.2320,0.431,0.1700,54,0.1880,2015,2015-11-13,7nLL5UrsXYHVWv2DnXBtud


In [27]:
# df.to_csv('../data_clean/spotify_kg_master.csv', index=False)

---

In [31]:
artists_df = pd.read_csv('../data_raw/kg_spotify_data_by_artist.csv')

In [32]:
artists_df.head()

,mode,count,acousticness,artists,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,9,0.590111,"""Cats"" 1981 Original London Cast",0.467222,250318.555556,0.394003,0.011400,0.290833,-14.448000,0.210389,117.518111,0.389500,38.333333,5
1,1,26,0.862538,"""Cats"" 1983 Broadway Cast",0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,30.576923,5
2,1,7,0.856571,"""Fiddler On The Roof” Motion Picture Chorus",0.348286,328920.000000,0.286571,0.024593,0.325786,-15.230714,0.118514,77.375857,0.354857,34.857143,0
3,1,27,0.884926,"""Fiddler On The Roof” Motion Picture Orchestra",0.425074,262890.962963,0.245770,0.073587,0.275481,-15.639370,0.123200,88.667630,0.372030,34.851852,0
4,1,7,0.510714,"""Joseph And The Amazing Technicolor Dreamcoat""...",0.467143,270436.142857,0.488286,0.009400,0.195000,-10.236714,0.098543,122.835857,0.482286,43.000000,5


In [33]:
artists_df.isnull().sum()

mode                0
count               0
acousticness        0
artists             0
danceability        0
duration_ms         0
energy              0
instrumentalness    0
liveness            0
loudness            0
speechiness         0
tempo               0
valence             0
popularity          0
key                 0
dtype: int64

In [34]:
artists_df.shape

(28680, 15)

In [44]:
artists_df[artists_df['artists'].str.contains('Cast', case=False)]

,mode,count,acousticness,artists,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,9,0.590111,"""Cats"" 1981 Original London Cast",0.467222,250318.555556,0.394003,0.011400,0.290833,-14.448000,0.210389,117.518111,0.389500,38.333333,5
1,1,26,0.862538,"""Cats"" 1983 Broadway Cast",0.441731,287280.000000,0.406808,0.081158,0.315215,-10.690000,0.176212,103.044154,0.268865,30.576923,5
4,1,7,0.510714,"""Joseph And The Amazing Technicolor Dreamcoat""...",0.467143,270436.142857,0.488286,0.009400,0.195000,-10.236714,0.098543,122.835857,0.482286,43.000000,5
5,1,36,0.609556,"""Joseph And The Amazing Technicolor Dreamcoat""...",0.487278,205091.944444,0.309906,0.004696,0.274767,-18.266389,0.098022,118.648944,0.441556,32.777778,5
88,0,2,0.113000,35MM: A Musical Exhibition Original Cast,0.772000,289587.000000,0.615000,0.000000,0.058100,-7.983000,0.063300,119.962000,0.704000,54.000000,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26908,1,2,0.170000,Verónica Castro,0.415000,139703.000000,0.781000,0.000000,0.040000,-8.895000,0.283000,191.835000,0.873000,20.000000,9
26975,1,19,0.068077,Victorious Cast,0.628211,182809.842105,0.880632,0.000001,0.172895,-3.449263,0.095716,127.523368,0.634737,53.473684,5
27187,1,4,0.447500,Waitress Original Broadway Cast Ensemble,0.678000,179640.000000,0.503000,0.000000,0.095450,-6.333500,0.047950,148.209500,0.681500,58.500000,7
27355,1,50,0.824080,West Side Story Original Broadway Cast,0.356168,186066.680000,0.263432,0.314513,0.394288,-19.932960,0.103612,109.203160,0.288864,17.120000,7
